<div class="alert alert-success">
<h3> Will be transated...</h3>

</div>

Нужно защитить данные клиентов страховой компании «Хоть потоп». Разработан метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснована корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## 1. Загрузка данных

In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

In [2]:
# Загружаем файл данных
try:
    data = pd.read_csv('https://С:/datasets/insurance.csv')
except:
    data = pd.read_csv(r'D:\Docs\Documents\ЯПрактикум\Проект7\insurance.csv')

In [3]:
# Посмотрим на исходные данные
display(data)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


## 2. Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** b. Качество модели линейной регрессии не изменится.

**Обоснование:** Умножение на обратимую матрицу является линейным преобразованием. Соответственно новые параметры линейной регрессии после преобразований будут линейно связаны со исходными параметрами. Формула обучения новой модели "учтет" изменения и новые параметры также будут соответсововать минимуму $MSE$. 

Обозначим  произведение исходной матрицы признаков $X$ на матрицу $P$ как $Z$. 

Тогда формула обучения будет следующая: $w_z = (Z^T Z)^{-1} Z^T y$, а предсказания: $a = Zw_z + w_0$. 

Можно перейти к сокращенной записи $a = Zw_z$, добавив первым столбцом матрицы $Z$ единичный столбец, а параметр $w_0$ добавив к вектору $w_z$.

## 3. Алгоритм преобразования

**Алгоритм**

Сгенерируем квадратную матрицу $P$ размерностью, равной размеру вектора признаков объекта `features.shape[1]` исходной матрицы признаков, заполненную случайными числами. Проверим, что полученная матрица обратима, выполнив операцию нахождения обратной матрицы. Умножим исходную матрицу признаков на созданную матрицу. Полученная матрица отличается от исходной, по ней сложно восстановить данные клиентов не имея инфорамции о матрице $P$.

**Обоснование**

Поскольку созданная матрица обратима, то мы можем заново обучить модель линейной регрессии для выполнения предсказаний, в соответствии с обоснованием, приведенным в пункте 2.

## 4. Проверка алгоритма

In [4]:
# Выделим признаки и целевой признак
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

In [5]:
# Создадим матрицу P, заполненную случайными числами от 0 до 10
P = np.random.randint(10, size=(features.shape[1], features.shape[1]))
P

array([[1, 7, 9, 2],
       [2, 5, 9, 6],
       [1, 9, 3, 3],
       [4, 3, 3, 1]])

In [6]:
# Проверим полученную матрицу на обратимость
np.linalg.inv(P)

array([[-0.07738095, -0.00595238, -0.0297619 ,  0.2797619 ],
       [ 0.0327381 , -0.07440476,  0.12797619, -0.00297619],
       [ 0.14186508,  0.0109127 , -0.11210317, -0.01289683],
       [-0.21428571,  0.21428571,  0.07142857, -0.07142857]])

In [7]:
# Напечатаем исходные признаки
print(features)

# Создадим класс модели линейной регрессии
class LinearRegression_class:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

# Создадим экземпляры модели линейной регрессии для библиотечной версии и описанной в классе
model = LinearRegression_class()
model_skl = LinearRegression()

# Обучим модели
model.fit(features, target)
model_skl.fit(features, target)

# Получим предсказания
predictions = model.predict(features)
predictions_skl = model_skl.predict(features)

# Напечатаем метрику R2 моделей
print('\nR2 модели из класса до преобразования:', r2_score(target, predictions))
print('R2 модели из библиотеки до преобразования:', r2_score(target, predictions_skl))

      Пол  Возраст  Зарплата  Члены семьи
0       1     41.0   49600.0            1
1       0     46.0   38000.0            1
2       0     29.0   21000.0            0
3       0     21.0   41700.0            2
4       1     28.0   26100.0            0
...   ...      ...       ...          ...
4995    0     28.0   35700.0            2
4996    0     34.0   52400.0            1
4997    0     20.0   33900.0            2
4998    1     22.0   32700.0            3
4999    1     28.0   40600.0            1

[5000 rows x 4 columns]

R2 модели из класса до преобразования: 0.4249455028666801
R2 модели из библиотеки до преобразования: 0.42494550286668


In [8]:
# Напечатаем измененные признаки
features = (data.drop('Страховые выплаты', axis=1)).dot(P)
print(features)

# Создадим экземпляры модели линейной регрессии для библиотечной версии и описанной в классе
model = LinearRegression_class()
model_skl = LinearRegression()

# Обучим модели с новыми признаками
model.fit(features, target)
model_skl.fit(features, target)

# Получим предсказания
predictions = model.predict(features)
predictions_skl = model_skl.predict(features)

# Напечатаем метрику R2 моделей
print('\nR2 модели из класса после преобразования:', r2_score(target, predictions))
print('R2 модели из библиотеки после преобразования:', r2_score(target, predictions_skl))

            0         1         2         3
0     49687.0  446615.0  149181.0  149049.0
1     38096.0  342233.0  114417.0  114277.0
2     21058.0  189145.0   63261.0   63174.0
3     41750.0  375411.0  125295.0  125228.0
4     26157.0  235047.0   78561.0   78470.0
...       ...       ...       ...       ...
4995  35764.0  321446.0  107358.0  107270.0
4996  52472.0  471773.0  157509.0  157405.0
4997  33948.0  305206.0  101886.0  101822.0
4998  32757.0  294426.0   98316.0   98237.0
4999  40661.0  365550.0  122064.0  121971.0

[5000 rows x 4 columns]

R2 модели из класса после преобразования: 0.4249455028666773
R2 модели из библиотеки после преобразования: 0.4249455028666753


## Выводы

- Преобразованные признаки существенно отличаются от исходных
- Метрика R2 не изменилась после произведенных преобразований (отличие в 12-м знаке после запятой связано с ограниченной точностью вычислительных методов).